In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# print(os.listdir('../data/reference_predictions'))
question_data = {}
forecast_data = {}

root_dir = '../data/reference_predictions'
for p in os.listdir(f'{root_dir}'):
    if(os.path.isdir(f'{root_dir}/{p}')):
        forecast_data[p[:15]] = pd.read_csv(f'{root_dir}/{p}/forecast_data.csv')
        question_data[p[:15]] = pd.read_csv(f'{root_dir}/{p}/question_data.csv')
def cdf_to_pdf(cdf, x=None):

    """Convert a CDF to a PDF via finite differences.

    Parameters
    ----------
    cdf : array-like, shape (n,)
        CDF values at grid points.
    x : array-like, shape (n,), optional
        Grid point locations. When provided, divides differences by bin
        widths so the result is a proper probability density. When omitted,
        returns probability masses (assumes unit spacing).

    Returns
    -------
    pdf : ndarray, shape (n,)
        PDF (or probability-mass) values aligned with the input grid.
    """
    cdf = np.asarray(cdf, dtype=float)
    mass = np.empty_like(cdf)
    mass[0] = cdf[0]
    mass[1:] = np.diff(cdf)

    if x is None:
        return mass

    x = np.asarray(x, dtype=float)
    dx = np.empty_like(x)
    dx[0] = 100 # x[1] - x[0]   # forward difference for first bin
    dx[1:] = np.diff(x)
    return mass / dx

eval_qid = []
eval_q = []
eval_bins = []
eval_pdf = []

for i, qid in enumerate(question_data.keys()):
    qd = question_data[qid]
    if( not qd['Question Type'].iloc[0] == 'numeric'):
        continue

    cr = qd['Continuous Range'].iloc[0]
    cr = np.array([float(x) for x in cr[1:-1].split(', ')])
    
    fd = forecast_data[qid]
    cdf_df = fd['Continuous CDF']
    cdf = np.array([float(x) for x in cdf_df.iloc[-1][1:-1].split(', ')])
    pdf = cdf_to_pdf(cdf, x=cr)
    eval_qid.append(qid)
    eval_q.append(qd['Question Title'].iloc[0])
    eval_bins.append(cr)
    eval_pdf.append(pdf)